In [70]:
import re
import pandas as pd
import sqlite3
def read_select_rules():
    query = f"""
    SELECT Component, [Project name], Version, Date, Owner, [OD rules], rules FROM T3_table
    WHERE Type = 'Select' AND [OD rules] IS NOT NULL AND [OD rules] LIKE '%Assertion%' """
    conn = sqlite3.connect('T3.db')
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    components = [row[0] for row in data if row[0] is not None]
    project_names = [row[1] for row in data if row[1] is not None]
    versions = [row[2] for row in data if row[2] is not None]
    dates = [row[3] for row in data if row[3] is not None]
    owners = [row[4] for row in data if row[4] is not None]
    OD_rules = [row[5] for row in data if row[5] is not None]
    rules = [row[6] for row in data if row[6] is not None]
    return components, project_names, versions, dates, owners, OD_rules, rules
components, project_name, version, date, owner, OD_rules, rules = read_select_rules()
OD_rules = [re.sub(r'\bSBB\w*\b', '', rule).strip() for rule in OD_rules]
newrules = [line.split(']')[0] for line in rules]
pattern = r'([✔✘])\s*(.*)'
data = []
data2 = []
section_id = 1 
for section in OD_rules:
    section = section.replace(':', '')
    section = section.replace('(', '')
    section = section.replace(')', '')
    items = re.split(r'\s*\,\s*', section)
    section_data = []
    if_data = []
    for item in items:
        if item.startswith("if"):
            sentence = item 
            sentence = sentence.replace('if', '') 
            item2s = re.split(r'\s*\|\|\s*|\s*&&\s*', sentence)
            for item2 in item2s:
                mat = re.match(pattern, item2)
                if match:
                    if_data.append((section_id, item2))
        else: 
            parts = re.split(r'\s*\|\|\s*|\s*&&\s*', item)
            for part in parts:
                match = re.match(pattern, part) 
                if match:
                    status, name = match.groups()
                    section_data.append((section_id, status, name))
    if section_data:
        data.extend(section_data)
    if if_data:
        data2.extend(if_data)
    section_id += 1
df1 = pd.DataFrame(data, columns=['Section_id', 'Status', 'name'])
df2 = pd.DataFrame(data2, columns=['Section_id', 'Rest'])
print(df2) 
OD_rules = [re.sub(r'[()]', '', rule).strip() for rule in OD_rules]

      Section_id                              Rest
0              1    ✔ RTX3050 4GB G6 128b DVI+H+DP
1              1      ✔ RTX3060 12GB G6 192b H+3DP
2              1     ✔ RX6500XT 4GB G6 64b H+DP HP
3              1    ✔ GTX1650 SP 4GB G6 128b 2DP+H
4              1  ✔ GTX1660SP 6GB G6 192b DVI+DP+H
...          ...                               ...
1461        1222         ✔  TSCA DP to HDMI Dongle
1462        1222          ✔  TSCA DP to VGA Dongle
1463        1222          ✔  TSCA DP to VGA Dongle
1464        1222     ✔  TSCA L1-VGA cable for Tiny
1465        1222     ✔  TSCA L1-VGA cable for Tiny

[1466 rows x 2 columns]


In [72]:
from py2neo import Graph, Node, Relationship
g = Graph('neo4j://localhost:7687', auth = ('neo4j', 'ys1203303'))
g.run("MATCH (n) DETACH DELETE n") 
for i in range(len(components)): 
    sec_df = df2[df2['Section_id'] == i + 1]  
    for a, aa in sec_df.iterrows(): 
        node1 = Node(
            "parent",
            Name = aa['Rest'],
            Component = components[i],
            originalRule = rules[i],
            Type = "Select",
            ProjectName = project_name[i],
            Version = version[i],
            owner = owner[i], 
            date = date[i],
            ruleIndex = OD_rules[i])
        g.create(node1) 
        section_df = df1[df1['Section_id'] == i + 1]  
        for s, ss in section_df.iterrows():
            node2 = Node(
                "son", 
                Name = ss['name'],
                Component = components[i],
                originalRule = rules[i],
                Type = "Select",
                ProjectName = project_name[i],
                Version = version[i],
                owner = owner[i], 
                date = date[i],
                ruleIndex = OD_rules[i])
            g.create(node2)
            if ss['Status'] == '✔':
                relationship = Relationship(node2, "should", node1) 
            else:
                relationship = Relationship(node2, "should not", node1) 
            g.create(relationship) 